In [ ]:
!pip install transformers

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 1.1MB 7.5MB/s 
     |████████████████████████████████| 3.0MB 23.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=7f27003561427ea348e19be643d7eae620ee594a501703908c7ebd6489972eff
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
Successfully built sentence-transformers
  Found existing installation: tokenizers 0.9.2
    Uninstalling tokenizers-0.9.2:
      Successfully uninstalled tokenizers-0.9.2
  Found existing installation: transformers 3.4.0
    Uninstalling transformers-3.4.0:
      Successfully uninstalled transformers-3.4.0


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import transformers

In [ ]:
text =  """ Supervised learning is the machine learning task of  learning a function that maps an input to an output based on example input-output pairs.[1] It infers a function from labeled training data consisting of a set of training examples.[2] In supervised learning, each example is a pair consisting of an input object (typically a vector) and a desired output value (also called the supervisory signal). A supervised learning algorithm analyzes the training data and produces an inferred function, which can be used for mapping new examples. An optimal scenario will allow for the algorithm to correctly determine the class labels for unseen instances. This requires the learning algorithm to generalize from the training data to unseen situations in a 'reasonable' way (see inductive bias).
      """
text = text.lower()

We first find the words/phrases that seem to be keywords/keyphrases

In [ ]:
!pip install pytextrank

In [ ]:
import spacy
import pytextrank

# example text
#text = "machine learning is the most important task in artificial inteligence so in order to train your model you should have a larg, very large dataset"


# load a spaCy model, depending on language, scale, etc.

def get_phrases(text,rate=0.05, printed=False):
    nlp = spacy.load("en_core_web_sm")

    # add PyTextRank to the spaCy pipeline
    tr = pytextrank.TextRank()
    nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

    doc = nlp(text)

    # examine the top-ranked phrases in the document
    if printed:
        for p in doc._.phrases:
            print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
            print(p.chunks)

    return [p.text for p in doc._.phrases if p.rank >= rate ]


In [ ]:

def XXX(text, candidates, rate=0.05):
    from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
    from sentence_transformers import SentenceTransformer
    from transformers import BertTokenizer
    
    #load the SBERT pre trained model
    sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

    #get the text embedding
    text_embedding = sbert_model.encode([text])

    #get the candidates embeddings
    candidates_embeddings = sbert_model.encode(candidates)

    #calcule the distances between the text and the candidates
    distances = cosine_similarity(text_embedding, candidates_embeddings)

    #sort and take the n_top candidates with low distances 
    
    keywords = [candidates[index] for index in distances.argsort()[0] if distances[0][index] >= rate]
    #keywords = [index for index in distances.argsort()[0][-n_top:]]

    return keywords


In [ ]:
text = "ESI is the is the international school of IT, it's the biggest school in algeria so far, you enter it with 18"

In [ ]:
candidates = get_phrases(text, rate=0.09)
print(candidates)

['algeria', 'the international school', 'the biggest school']


In [ ]:
keywords = XXX(text, candidates, rate=0.1)
print(keywords)

100%|██████████| 405M/405M [00:17<00:00, 23.2MB/s]


['algeria', 'the international school', 'the biggest school']


In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('treebank')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [4]:
text = """The science of physiology, which is concerned with the normal functions of the animal body, is the basis of medicine and surgery, which are concerned with the abnormal functions of the human body."""
tokens = nltk.word_tokenize(text)
tagged = nltk.pos_tag(tokens)
print(tagged)
entities = nltk.chunk.ne_chunk(tagged)
print(entities)

[('The', 'DT'), ('science', 'NN'), ('of', 'IN'), ('physiology', 'NN'), (',', ','), ('which', 'WDT'), ('is', 'VBZ'), ('concerned', 'VBN'), ('with', 'IN'), ('the', 'DT'), ('normal', 'JJ'), ('functions', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('animal', 'NN'), ('body', 'NN'), (',', ','), ('is', 'VBZ'), ('the', 'DT'), ('basis', 'NN'), ('of', 'IN'), ('medicine', 'NN'), ('and', 'CC'), ('surgery', 'NN'), (',', ','), ('which', 'WDT'), ('are', 'VBP'), ('concerned', 'VBN'), ('with', 'IN'), ('the', 'DT'), ('abnormal', 'JJ'), ('functions', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('human', 'JJ'), ('body', 'NN'), ('.', '.')]
(S
  The/DT
  science/NN
  of/IN
  physiology/NN
  ,/,
  which/WDT
  is/VBZ
  concerned/VBN
  with/IN
  the/DT
  normal/JJ
  functions/NNS
  of/IN
  the/DT
  animal/NN
  body/NN
  ,/,
  is/VBZ
  the/DT
  basis/NN
  of/IN
  medicine/NN
  and/CC
  surgery/NN
  ,/,
  which/WDT
  are/VBP
  concerned/VBN
  with/IN
  the/DT
  abnormal/JJ
  functions/NNS
  of/IN
  the/DT
  human/JJ
  body/N